<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC_0905%EC%B6%94%EA%B0%80_%EC%97%AD%EB%B2%88%EC%97%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [ ]:
# !pip install konlpy

In [ ]:
# !pip install transformers

In [ ]:
# !pip install optuna

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

In [ ]:
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/project_final/data/"
DATA_PATH = "C:\python-code\project_final\data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 데이터 불러오기

- 0905 추가 데이터 + 텍스트 정제 + 가게명/음식명 삭제 + 오버샘플링(paraphrased) + 맞춤법 검사 반영된 데이터

In [ ]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean_remove_over.csv")
train

,review,reply,star
0,저도 모르게 은근 여기 자주 시켜 먹고 있었네요 늘 잘 먹고 있습니다,저희 메뉴 맛있죠 이렇게 빠져드시는 거예요 그냥 일로서 만드는 게 아니라 진짜 제 ...,5
1,김치찌개 된장찌개 잡채 다 잘 먹었습니다,전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...,5
2,처음 시켜보는데 맛있고 리뷰 서비스도 괜찮네요 다음에도 또 시킬 듯요,안녕하세요 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기 망...,5
3,맛있습니다 근데 국물이 안 와서 아쉽습니다,안녕하세요 저희 측 실수로 인하여 누락이 되었군요 이에 대해서 사과 말씀드립니다 다...,4
4,맛있게 잘 먹었습니다,헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요,5
...,...,...,...
26162,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,맛있는 식사가 되셨기를 바랍니다 맛있는 식사 이용해 주셔서 감사합니다,4
26163,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락 주시면 ...,4
26164,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4
26165,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해 본 결과 ...,4


In [ ]:
!pip install translate

In [ ]:
from translate import Translator

ko = "진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 장난하는것도아니고"
translator_ko2en = Translator(to_lang="en", from_lang="ko")
ko2en = translator_ko2en.translate(ko)

ko2en

"I don't really use reviews, but the amount costs 20,000 won and it's never the amount to eat. I'm not kidding."

In [ ]:
translator_en2ko = Translator(to_lang="ko", from_lang="en")
en2ko = translator_en2ko.translate(ko2en)

en2ko

'후기를 잘 안쓰는데 금액이 2만원이고 절대 먹을 금액이 아닙니다. 농담이 아닙니다.'

In [ ]:
print(train['review'][0])
review_ko2en = translator_ko2en.translate(train['review'][0])
print(review_ko2en)
review_en2ko = translator_en2ko.translate(review_ko2en)
print(review_en2ko)

저도 모르게 은근 여기 자주 시켜 먹고 있었네요 늘 잘 먹고 있습니다
I was eating here a lot without realizing it. I'm always eating well.
저도 모르게 여기서 많이 먹고 있었어요. 항상 잘 먹고 있습니다.


In [ ]:
# 함수화
from translate import Translator
translator_ko2en = Translator(to_lang="en", from_lang="ko")
translator_en2ko = Translator(to_lang="ko", from_lang="en")

def back_translation(text):
    ko2en = translator_ko2en.translate(text)
    en2ko = translator_en2ko.translate(ko2en)
    return en2ko

In [ ]:
star_1 = train[train['star'] == 1]
star_2 = train[train['star'] == 2]
star_3 = train[train['star'] == 3]
star_4 = train[train['star'] == 4]

train_star = pd.concat([star_1, star_2, star_3, star_4], ignore_index=True)
train_star

,review,reply,star
0,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다 배달이 20분 넘게 안 오길래 ...,안녕하세요 주신 리뷰 잘 읽었습니다 뒤돌아 보니 저희 응대에 미흡했던 이 있던 것 ...,1
1,만들어 놓은지 한참 된 커피 배달 예정 시간 25분이 지났어요,안녕하세요 배달이 늦어져 죄송합니다 한파로 길이 얼어서 고객님들이 상황이 여유치가 ...,1
2,,사이드만 잔뜩 시키고 1,1
3,밥이 쉬었어요 단무지 때문도 아니고 김이 많아서 그런 것도 아니고 쉬어서 냄새가 나...,밥이 쉬다뇨 말도 안 되는 소리 좀 하지 마세요 진짜 먼 정신이 이상한 사람인가 하...,1
4,잠실점에 처음 시켜 먹었던 게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 들어와...,요기요가 주문 앱에 렉이 자주 있어서 여러 가지로 혼돈이 있어 정말 죄송하게 되었습...,1
...,...,...,...
2383,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,맛있는 식사가 되셨기를 바랍니다 맛있는 식사 이용해 주셔서 감사합니다,4
2384,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락 주시면 ...,4
2385,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4
2386,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해 본 결과 ...,4


In [ ]:
tqdm.pandas()
over_review = train_star['review'].progress_apply(back_translation)
over_review

  0%|          | 0/2388 [00:00<?, ?it/s]

In [ ]:
tqdm.pandas()
over_reply = train_star['reply'].progress_apply(back_translation)
over_reply

In [ ]:
train_over_review = pd.DataFrame({"review":over_review})
train_over_reply = pd.DataFrame({"reply":over_reply})

# concat
train_over = pd.concat([train_over_review, train_over_reply, train_star["star"]], axis=1)
train_over

- 저장

In [ ]:
train_over.to_csv(f'{DATA_PATH}train_over_backtrans.csv', index=False, encoding = "utf-8-sig")

- 불러오기

In [ ]:
train_over = pd.read_csv(f'{DATA_PATH}train_over_backtrans.csv')
train_over